# Exercise set 2 


In [27]:
import numpy as np
import matplotlib.pyplot as plt

## Exercise 1: Newton Coates


The main task of this problem set is the following: for a given interval $(a,b)$ and $n+1$ equally distributed nodes $x_i = a + i \tfrac{(b-a)}{n}$
for $i=0,\ldots n$, tabulate the weights for the Newton-Cotes formula up to $n=14$.
The problem boils down to 2 task, namely 

1. Defining the Lagrange polynomials $L_{in}$ for $i=0, \ldots, n$
2. Computing the weights $ w_i = \int_a^b L_{in}(x) \,\mathrm{d}x $

__Before you start:__ For the implementation of the task, we recommend to use the [sympy](https://docs.sympy.org/latest/index.html#) python module  for symbolic mathematics to perform tasks such as (symbolic) integration.
Spend some time to browse  through the  [sympy tutorial](https://docs.sympy.org/latest/tutorial/index.html) and the [help on symbolic integration](https://docs.sympy.org/latest/modules/integrals/integrals.html). Using ```sympy```__Before you start:__ For the implementation of the task, we recommend to use the [sympy](https://docs.sympy.org/latest/index.html#) python module  for symbolic mathematics to perform tasks such as (symbolic) integration.
Spend some time to browse  through the  [sympy tutorial](https://docs.sympy.org/latest/tutorial/index.html) and the [help on symbolic integration](https://docs.sympy.org/latest/modules/integrals/integrals.html). Using ```sympy``` you can for instance do something like this for $n=2$:


In [28]:
# import symbol x from sympy so that you can define symbolic functions of x
from sympy.abc import x
# import symbolic integration
from sympy import integrate 
from functools import reduce

# Define
a, b = 0, 1 
xqs = np.linspace(a,b,3)
# Define L_02 (not normalized)
L_02 = (x-xqs[1])*(x-xqs[2])
# Normalize it to satisfy l_02(x_0) = 1
L_02 = L_02/L_02.subs(x,xqs[0])

# Now integrate L_02 to compute the first weight
w_0 = integrate(L_02, (x, a, b))
print("w0 = {}".format(w_0))

w0 = 0.166666666666667


Of course, 
since you are asked to do comupute all $n+1$ weights  $n=1,\ldots,14$,
you need to automatize the construction of the corresponding Lagrange polynoms.
So proceed as follows:

__a__) Write  a python function ```lagrange_polys``` which takes a list of $n+1$ quadrature points
and returns a list of the corresponding $n+1$ Lagrange polynoms $\{L_{in}\}_{i=0}^n$
defined a symbolic function using ```sympy```:

In [29]:
import sympy
from functools import reduce
import numpy as np

x = sympy.symbols('x')

def lagrange_polys(xqs):
    return [reduce(lambda x,y:x*y, [(x-xqs[j])/(xqs[i]-xqs[j]) for j in range(len(xqs)) if j != i]) for i in range(len(xqs))]

__b__) Now the easy part! Employ your brand new ```def lagrange_polys``` function and implement a python function which takes as argument the desired degree of exactness $n$ and the interval end points
$a,b$ and returns a list of quadrature points $\{x_i\}_{i=0}^n$ and quadrature weights $\{w_i\}_{i=0}^n$:

In [30]:
def newton_cotes(n, a, b):
    xqs = np.linspace(a, b, n)
    ls = lagrange_polys(xqs)
    ws = [sympy.integrate(l, (x, a, b)) for l in ls]
    return np.array(xqs), np.array(ws)


__c__) Before you tabulate the quadrature weights with you newly implemented function, make sure that you implement them correctly. More, specifically, check for $n=1,\ldots 14$
that the computed Newton-Cotes formula has the expected degree of exactness, meaning that it integrates polynomials up to order $n$ __exactly__.

For $n$ is even, check that the corresponding Newton-Cotes rules even integrate polynomials up to order $n+1$  exactly (and not only up to $n$).

Note:  Due to floating point related errors and some numerical instabilities when computing
higher order Lagrange polynomials and integrals, the difference between the exact integral 
and the numerically error won't be 0, but around the machine precision for $n=1,2$ and then
for each increase of the order $n$ you will roughly loose of significant digit in
the difference between the two.



In [31]:
def QR(f, xq, wq):
    return wq@f(xq)

m = 14
xq, wq = newton_cotes(m, 0, 1)
for n in range(1, m+3):
    f = lambda x_: x_**n
    print(f"n: {n}", abs(1/(n+1) - QR(f, xq, wq)))




n: 1 6.42920050530904e-9
n: 2 5.14046100130727e-9
n: 3 3.95593885427203e-9
n: 4 2.77866807341098e-9
n: 5 1.77837433668060e-9
n: 6 1.02457375916742e-9
n: 7 5.03083630309931e-10
n: 8 1.66615471419718e-10
n: 9 3.57307239351456e-11
n: 10 1.46744311146918e-10
n: 11 1.98536978524011e-10
n: 12 2.13662892933897e-10
n: 13 2.07264538865104e-10
n: 14 1.28465678095324e-8
n: 15 9.76024299575995e-8
n: 16 4.02676261897217e-7


__d__) Tabulate the quadrature weights for the Newton-Cotes rule for $n=1,\ldots 14$. For which $n$  should you
refrain from using the resulting quadrature rule (and why?)

In [36]:
for n in range(2, 15):
    xq, wq = newton_cotes(n, 0, 1)
    if [i for i in wq if i < 0]:
        print(f"neg weigths, n={n}")

neg weigths, n=9
neg weigths, n=11
neg weigths, n=12
neg weigths, n=13
neg weigths, n=14


## Exercise 2: composite Simpson's rule
Simpson's rule is defined as
$$
S[f](x_{i-1}, x_i) = \frac{h}{6}(f(x_{i-1} + 4f(x_{i-1/2}) + f(x_i))
$$
where $h = x_i - x_{i-1}$ and $x_{i-1/2} = \frac{x_{i-1}+x_i}{2}.$


**a)**

Show that the resulting composite Simpson's rule is given by

\begin{align*}
\int_a^b f {\,\mathrm{d}x} \approx \mathrm{CSR}[f]({[x_{i-1}, x_i]}_{i=1}^{m})
&= 
\tfrac{h}{6}
[
f(x_0)
+ 4f(x_{x_{1/2}}) + 2f(x_1) 
+ 4f(x_{3/2}) +     2f(x_2)
+ \ldots
\\ 
&\qquad+2 f(x_{m-1})
+
4f(x_{x_{m-1/2}}) 
+f(x_m)
].
\end{align*}

**b)**
Implement the composite Simpson's rule.
Use this function to compute an approximated value of the integral

$$
I(0,1) = \int_0^1 \tan\left(\frac{\pi}{4}x\right) = 2\frac{\log(2)}{\pi} = 0.4412712\dotsc.
$$

for $m = 4, 8, 16, 32, 64$ corresponding to
$ h = 2^{-2}, 2^{-3}, 2^{-4}, 2^{-5}, 2^{-6}$.
Tabulate the corresponding quadrature errors $|I(0,1) - \mathrm{CSR}[f]({[x_{i-1}, x_i]}_{i=1}^{m})|$. Plot the errors against $h$ in a $\log-\log$
plot and determine the EOC ("Experimental Order of Convergence")
How does it compare to the composite trapezoidal rule?

**c)**
Recall that the error of Simpson's rule on a single interval is given by

$$
|I[f](a,b) - S[f](a,b)| = - \frac{(b-a)^5}{2880}  f^{(4)}(\xi)
$$
for some $\xi \in [a,b]$.

Use this to show that the error of the composite Simpson rule can be bounded by

\begin{equation}
|I[f]-\mathrm{CSR}[f]|
\leqslant
\dfrac{M_4}{2880} \dfrac{(b-a)^5}{m^4}
=
\dfrac{M_4}{2880}
h^4(b-a)
\end{equation}

where  $M_4 = \max_{\xi\in[a,b]} |f^{(4)}(\xi)|$. Does your numerical experiments from b) support the theoretically derived convergence order?

**d)** Redo the numerical experiment from b), but this time, use the composite Simpson rule to 
compute approximated values of the integral

$$
\int_0^1 \sqrt{1-x^2} dx = \dfrac{\pi}{4}.
$$
What EOC do you obtain? Do you have an explanation for reduced convergence order?

**Hint.** Have a look a $f'$, e.g. by plotting $f'$ over intervals $[0,b]$ with $0 < b < 1$ but $b$ very close to $1$, e.g. $b=0.9999$  

## Exercise 3: composite Gauss-Legandre

Gauss-Legandre on the interval $[-1,1]$ is defined as 
$$
GL[f](-1, 1) =f(1/\sqrt{3}) + f(-1/\sqrt{3})
$$


__a__) Transfer the quadrature points and weights to an arbitrary interval $[x_{i-1},x_{i}]$, and show that the composite Gauss-Legandre rule is given by: 


\begin{align*}
\int_a^b f {\,\mathrm{d}x} \approx \mathrm{CGL}[f]({[x_{i-1}, x_i]}_{i=1}^{m})
&= 
\frac{h}{2}\sum_{i=1}^m \left( f\left(\frac{x_i + x_{i-1}}{2} + \frac{x_i - x_{i-1}}{2\sqrt{3}}\right) +  f\left(\frac{x_i + x_{i-1}}{2} - \frac{x_i - x_{i-1}}{2\sqrt{3}} \right) \right) .
\end{align*}

where $h = x_i - x_{i-1}$

__b__) Implement the composite Gauss-Legandre and determine the EOC as you did in exercise 2b)